In [1]:
from hhop import DFExtender, write_table_through_view, spark, col, F

22/12/15 21:06:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/15 21:06:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Возможно, пригодятся 
1. .config("spark.hadoop.hive.exec.dynamic.partition", "true") \
2. .config("spark.hadoop.hive.exec.dynamic.partition.mode", "nonstrict") \

## Synth table

In [7]:
df_src = spark.read.csv('./synth_data/table1.csv', header=True, sep=';')

In [8]:
df_src.show()

+-----+----+----+------+--------+----------+----------+
|index| pk1| pk2|  var1|    var2|   dt_part|group_part|
+-----+----+----+------+--------+----------+----------+
|    1|key1|   1|value1|value2_1|2022-12-15|    group1|
|    2|key1|   1|value1|value2_1|2022-12-16|    group2|
|    3|key1|   2|value1|value2_1|2022-12-16|    group3|
|    4|key2|   2|  null|value2_1|2022-12-17|    group1|
|    5|key2|   3|value1|value2_1|2022-12-18|    group2|
|    6|key2|   4|value1|    null|2022-12-19|    group3|
|    7|key2|null|value1|value2_1|2022-12-19|    group4|
|    8|null|   4|value1|value2_1|2022-12-20|    group3|
|    9|null|null|value1|value2_1|2022-12-20|    group7|
+-----+----+----+------+--------+----------+----------+



In [9]:
df_src_write = df_src.write.mode('overwrite')

df_src_write.partitionBy('dt_part', 'group_part').saveAsTable('default.part_table_test1')
df_src_write.saveAsTable('default.nonpart_table_test1')

## Info about table

In [10]:
df = DFExtender('default','part_table_test1', pk=['pk1', 'pk2'])

table info default.part_table_test1
----------------------------------------
root
 |-- index: string (nullable = true)
 |-- pk1: string (nullable = true)
 |-- pk2: string (nullable = true)
 |-- var1: string (nullable = true)
 |-- var2: string (nullable = true)
 |-- dt_part: string (nullable = true)
 |-- group_part: string (nullable = true)

cols: ['index', 'pk1', 'pk2', 'var1', 'var2', 'dt_part', 'group_part']
partition columns: ['dt_part', 'group_part']
see attribute like this for full info: DataFrame.describe_table.show(100, False)

location: file:/Users/pyro/github/HiveHelper_on_PySpark/spark-warehouse/part_table_test1


In [13]:
df.describe_table.show(3, 30)

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|   index|   string|   null|
|     pk1|   string|   null|
|     pk2|   string|   null|
+--------+---------+-------+
only showing top 3 rows



## SchemaManager

## Comparing tables